# Random Forest Regressor from Scratch
***
## Table of Contents
1. [Introduction](#1-introduction)
***

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Optional
from numpy.typing import NDArray

## 1. Introduction
This notebook is an extension of [Decision Tree Regressor from Scratch](https://github.com/tsu76i/DS-playground/blob/main/2.%20Building%20ML%20Models%20From%20Scratch/2.3%20CART/decision_tree_regressor.ipynb).

Random forests are an ensemble learning technique that combines multiple decision trees, each trained on a random subset of the data (with replacement) and a random subset of features at each split. The final prediction is made by aggregating the results of all trees(**majority vote** for classification, **average** for regression). Compared to decision trees, this approach provides better accuracy, reduced overfitting and more stable predictions, though at the cost of increased computational complexity and reduced interpretability. This method introduces two key randomisation techniques during the training process:

1. **Bootstrap Sampling**: Each tree is trained on a bootstrapped dataset, which is a random sample of the original dataset created *with replacement*. This ensures diversity among the trees.
2. **Feature Randomisation**: At each split in a tree, a random subset of features is considered rather than evaluating all features. This prevents dominant features from appearing in every tree and further promotes diversity.

## 2. Loading Data
Retrieved from [GitHub - YBI Foundation](https://github.com/YBI-Foundation/Dataset/blob/main/Admission%20Chance.csv)

In [2]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Admission%20Chance.csv')
df.head()

,Serial No,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
feature_names = df.columns[:-1].tolist()  # All columns except the last one

# Check the shape of the data
print(f'Features shape: {X.shape}')
print(f'Target shape: {y.shape}')
print(f'Features: \n{feature_names}')

Features shape: (400, 8)
Target shape: (400,)
Features: 
['Serial No', 'GRE Score', 'TOEFL Score', 'University Rating', ' SOP', 'LOR ', 'CGPA', 'Research']


## 3. Train Test Split
Train test split is a fundamental model validation technique in machine learning. It divides a dataset into two separate portions: a **training set** used to train a model, and a **testing set** used to evaluate how well the model can perform on unseen data. 

The typical split ratio is 80% for training and 20% for testing, though this can vary (70/30 or 90/10 are also common). The key principle is that the test set must remain completely separated during model training process, and should never be used to make decisions about the model or tune parameters. 

The split is usually done randomly to ensure both sets are representative of the overall dataset, and many libraries (such as scikit-learn) provide build-in functions that handle this process automatically while maintaining proper randomisation.


In [4]:
def train_test_split(X: pd.DataFrame, y: pd.Series, test_size: float = 0.2,
                     random_state: int = None) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    """
    Split arrays or matrices into random train and test subsets.

    Args:
        X: Input features, a 2D array with rows (samples) and columns (features).
        y: Target values/labels, a 1D array with rows (samples).
        test_size: Proportion of the dataset to include in the test split. Must be between 0.0 and 1.0. default = 0.2
        random_state: Seed for the random number generator to ensure reproducible results. default = None

    Returns:
        A tuple containing:
            - X_train: Training set features.
            - X_test: Testing set features.
            - y_train: Training set target values.
            - y_test: Testing set target values.
    """
    # Set a random seed if it exists
    if random_state:
        np.random.seed(random_state)

    # Create a list of numbers from 0 to len(X)
    indices = np.arange(len(X))

    # Shuffle the indices
    np.random.shuffle(indices)

    # Define the size of our test data from len(X)
    test_size = int(test_size * len(X))

    # Generate indices for test and train data
    test_indices: NDArray[np.int64] = indices[:test_size]
    train_indices: NDArray[np.int64] = indices[test_size:]

    # Return: X_train, X_test, y_train, y_test
    return X.iloc[train_indices], X.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

## 4. Loss Functions for Regression
### Variance

In [5]:
def variance(y: pd.Series) -> float:
    return np.var(y)

### Mean Squared Error (MSE) For LF

In [6]:
def mse(y: pd.Series) -> float:
    mean = np.mean(y)
    return np.mean((y - mean) ** 2)

In [7]:
print(f"Variance: {variance(y):.5f}")
print(f"MSE: {mse(y):.5f}")

Variance: 0.02029
MSE: 0.02029


## 5. Information Gain
Information Gain is a metric used to measure the effectiveness of a feature in splitting a dataset into subsets that are more pure concerning the target variable. It quantifies the reduction in variance or MSE, and a higher information gain indicates a better feature for making splits.

\begin{align*}
IG(S, A) = H(S) - \sum_{i=1}^{n} \dfrac{|S_i|}{|S|}H(S_{i})
\end{align*}

where:
- $H(S)$: Variance (or MSE) of the original dataset $S$.
- $S_{i}$: Subset of $S$ created by splitting on feature $A$ for the $i_{th}$ value or range of the feature.
- $\dfrac{|S_i|}{|S|}$: Proportion of samples in subset $S_{i}$.
- $H(S_{i})$: Variance (or MSE) of subset $S_{i}$.



The following `information_gain` function calculates the difference between the metric for the parent node and the weighted average of the metrics for the child nodes (left and right splits).

In [8]:
def information_gain(y: pd.Series, y_left: pd.Series, y_right: pd.Series,
                     metric: str = 'variance') -> float:
    """
    Calculate the information gain for regression.

    Args:
        y: Target variables of the parent node.
        y_left: Target variables of the left child node after the split.
        y_right: Target variables of the right child node after the split.
        metric: Splitting criterion, either 'variance' or 'mse'. Defaults to 'variance'.

    Returns:
        Information gain resulting from the split.
    """
    if metric == 'variance':
        parent_metric = variance(y)
        left_metric = variance(y_left)
        right_metric = variance(y_right)
    else:  # metric == "mse"
        parent_metric = mse(y)
        left_metric = mse(y_left)
        right_metric = mse(y_right)

    weighted_metric = (
        len(y_left) / len(y) * left_metric
        + len(y_right) / len(y) * right_metric
    )
    return parent_metric - weighted_metric

## 6. Bootstrapping
Bootstrapping is a statistical resampling method that involves sampling data points with replacement. In creating a new dataset (**bootstrap sample**) from the original dataset, some data points may appear multiple times, while others may be excluded. Though individual data points may repeat, the size of bootstrap sample $n$ is typically the same as the original dataset. This method ensures variability among datasets, which helps reduce overfitting when used in ensemble learning.

For a dataset with $n$ examples, each sample has a $1 - \left( 1 - \dfrac{1}{n} \right)^{n}$ chance of being selected at least once in the bootstrap sample. As $n$ becomes large, this value approaches $1-\text{e}^{-1} \approx 0.632$. Hence, about 63.2% of the original dataset is expected to appear in any given bootstrap sample.

In [ ]:
def bootstrap_sample(X: pd.DataFrame, y: pd.Series, n_samples: Optional[int] = None,
                     random_state: Optional[int] = None) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Generate a bootstrap sample from the dataset.

    Args:
        X: Input features.
        y: Target labels.
        n_samples: Samples to draw (default: dataset size).
        random_state: Random seed.

    Returns:
        Bootstrapped (X, y) tuple.
    """
    if random_state is not None:
        np.random.seed(random_state)
    if n_samples is None:
        n_samples = len(X)
    indices = np.random.randint(0, len(X), size=n_samples)
    return X.iloc[indices], y.iloc[indices]

In [10]:
bootstrap_sample(X, y, random_state=42)[0][:10]  # X

,Serial No,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
102,103,314,106,2,4.0,3.5,8.25,0
348,349,302,99,1,2.0,2.0,7.25,0
270,271,306,105,2,2.5,3.0,8.22,1
106,107,329,111,4,4.5,4.5,9.18,1
71,72,336,112,5,5.0,5.0,9.76,1
188,189,331,115,5,4.5,3.5,9.36,1
20,21,312,107,3,3.0,2.0,7.90,1
102,103,314,106,2,4.0,3.5,8.25,0
121,122,334,119,5,4.5,4.5,9.48,1
214,215,331,117,4,4.5,5.0,9.42,1


In [11]:
bootstrap_sample(X, y, random_state=42)[1][:10]

102    0.62
348    0.57
270    0.72
106    0.87
71     0.96
188    0.93
20     0.64
102    0.62
121    0.94
214    0.94
Name: Chance of Admit , dtype: float64